In [ ]:
import time

import IPython.display as ipd
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import utils

In [ ]:
tracks = utils.load('fma_metadata/tracks.csv')
features = utils.load('fma_metadata/features.csv')
echonest = utils.load('fma_metadata/echonest.csv')

tracks.shape, features.shape, echonest.shape

((106574, 52), (106574, 518), (13129, 249))

In [3]:
echonest

echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   
...                 ...          ...       ...              ...       ...   
124857         0.007592     0.790364  0.719288         0.853114  0.720715   
124862         0.041498     0.843077  0.536496         0.865151  0.547949   
124863         0.000124     0.609686  0.895136         0.846624  0.632903   
124864         0.327576     0.574426  0.548327         0.452867  0.075928   
124911         0.993606     0.499339  0.050622         0.945677  0.095965   

                                                                          ...  \
                                           metadata                       ...   
         speechiness    tempo   valence  album_date           album_name  ...   
track_id                                                                  ...   
2           0.159310  165.922  0.576661         NaN                  NaN  ...   
3           0.461818  126.957  0.269240         NaN                  NaN  ...   
5           0.124595  100.260  0.621661         NaN                  NaN  ...   
10          0.032985  111.562  0.963590  2008-03-11    Constant Hitmaker  ...   
134         0.525519  114.290  0.894072         NaN                  NaN  ...   
...              ...      ...       ...         ...                  ...  ...   
124857      0.082550  141.332  0.890461         NaN                  NaN  ...   
124862      0.074001  101.975  0.476845         NaN                  NaN  ...   
124863      0.051517  129.996  0.496667         NaN                  NaN  ...   
124864      0.033388  142.009  0.569274         NaN                  NaN  ...   
124911      0.065189  119.965  0.204652  2009-10-23  Suicide Beauty Girl  ...   

                                                                              \
         temporal_features                                                     
                       214        215       216       217       218      219   
track_id                                                                       
2                -1.992303   6.805694  0.233070  0.192880  0.027455  0.06408   
3                -1.582331   8.889308  0.258464  0.220905  0.081368  0.06413   
5                -2.288358  11.527109  0.256821  0.237820  0.060122  0.06014   
10               -3.662988  21.508228  0.283352  0.267070  0.125704  0.08082   
134              -1.452696   2.356398  0.234686  0.199550  0.149332  0.06440   
...                    ...        ...       ...       ...       ...      ...   
124857           -1.721207   4.686078  0.213789  0.208800  0.007911  0.06395   
124862           -0.647897   1.282306  0.214586  0.181860  0.011247  0.06240   
124863           -0.771613   1.623510  0.180471  0.128185  0.010103  0.06222   
124864           -2.054143   7.927149  0.250178  0.219205  0.014851  0.06390   
124911           -0.215639  -0.584081  0.603893  0.505940  0.608585  0.06830   

                                                      
                                                      
                220       221        222         223  
track_id                                              
2          3.676960   3.61288  13.316690  262.929749  
3          6.082770   6.01864  16.673548  325.581085  
5          5.926490   5.86635  16.013849  356.755737  
10         8.414010   8.33319  21.317064  483.403809  
13

In [ ]:
def test_classifiers_features(classifiers, data, labels):
    columns = ["dim"] + list(classifiers.keys())
    scores = pd.DataFrame(columns=columns, index=["echonest_all"])
    times = pd.DataFrame(columns=classifiers.keys())
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
    scores['dim'] = X_train.shape[1]
    for clf_name, clf in classifiers.items():
        print(clf_name, "====================start")
        t = time.process_time()
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print(score)
        scores[clf_name] = score
        times[clf_name] = time.process_time() - t
        print(clf_name, "====================end")
    return scores, times

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

In [8]:
label = tracks["track", "genre_top"].dropna().reset_index()
label.columns = [''.join(col) for col in label.columns]
label = label.rename(columns={"trackgenre_top" : "genre"})

In [9]:
echonest_temporal_data = echonest["echonest", "temporal_features"]
echonest_temporal_data = echonest_temporal_data.merge(label, how="inner", on="track_id")
echonest_temporal_data = echonest_temporal_data.drop(columns="track_id")

In [10]:
label = echonest_temporal_data["genre"]
echonest_temporal_data = echonest_temporal_data.drop(columns="genre")

In [ ]:
rstate= 42

classifiers = {
   'LR': LogisticRegression(n_jobs=-1, random_state=rstate),
   'kNN': KNeighborsClassifier(n_neighbors=200, n_jobs=-1),
   'SVCrbf': SVC(kernel='rbf', random_state=rstate),
   'DT': DecisionTreeClassifier(max_depth=100),
   'RF1': RandomForestClassifier(max_depth=20, n_estimators=20),
}


scores, times = test_classifiers_features(classifiers, echonest_temporal_data, label)


LR ====================start


/opt/homebrew/Caskroom/miniconda/base/envs/ml_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6323215049166311
LR ====================end
kNN ====================start
0.5083368961094484
kNN ====================end
SVCrbf ====================start
0.5814450619923044
SVCrbf ====================end
DT ====================start
0.5664814023086789
DT ====================end
RF1 ====================start
0.6883283454467721
RF1 ====================end


In [ ]:
ipd.display(format_scores(scores))

,dim,LR,kNN,SVCrbf,DT,RF1
echonest_all,224,63.23%,50.83%,58.14%,56.65%,68.83%


,LR,kNN,SVCrbf,DT,RF1


In [13]:
scores.to_csv('scores_.csv', index=False)  # Set index=True if you want to include the index column

# Save the `times` table as a CSV file
times.to_csv('times_.csv', index=False)